In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.strategies.strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from quantfreedom.simulate import sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
import requests

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=3000)

It took 00 mins and 01 seconds to download 3000 candles


In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
# backtest_settings = BacktestSettings(
#     divide_records_array_size_by=1,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5, 6]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1]),
    trail_sl_when_pct=np.array([1, .5]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [4]:
strat_records_df = sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.File,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 24
Total combinations of settings to test: 144
Total candles: 3,000
Total candles to test: 432,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,70.0,24,46,-5.62,34.29,-0.2064,1570.062,-56.2067,943.7933
1,0,1,97.0,28,69,-48.72,28.87,-0.0000,2067.028,-487.1761,512.8239
2,0,2,72.0,23,49,-16.75,31.94,-0.4023,1775.831,-167.5180,832.4820
3,0,3,87.0,25,62,-37.41,28.74,-0.0000,2082.074,-374.0844,625.9156
4,0,4,72.0,18,54,-32.85,25.00,-0.0087,2762.934,-328.5113,671.4887
...,...,...,...,...,...,...,...,...,...,...,...
139,5,19,9.0,3,6,25.75,33.33,0.0000,259.678,257.4940,1257.4940
140,5,20,9.0,4,5,36.51,44.44,0.0000,262.079,365.0916,1365.0916
141,5,21,9.0,4,5,32.63,44.44,0.0000,250.404,326.2557,1326.2557
142,5,22,9.0,3,6,23.47,33.33,0.0000,253.281,234.6616,1234.6616


In [5]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

In [6]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,0,100,1697713500000,2023-10-19 11:05:00,EntryFilled,1000.0000,924.2260,5599.585,75.774,...,5675.359,28431.0,NaN,0.200,5675.359,NaN,0.35,28331.8,1.2,28765.8
1,0,0,101,1697713800000,2023-10-19 11:10:00,EntryFilled,1000.0000,850.3070,10964.530,149.693,...,5514.638,28435.4,NaN,0.394,11189.997,NaN,0.35,28331.8,1.2,28771.6
2,0,0,102,1697714100000,2023-10-19 11:15:00,EntryFilled,1000.0000,768.0630,16924.877,231.937,...,6192.284,28421.3,NaN,0.612,17382.281,NaN,0.34,28332.4,1.2,28758.2
3,0,0,147,1697727600000,2023-10-19 15:00:00,TakeProfitFilled,1180.1642,1180.1642,NaN,NaN,...,NaN,NaN,28758.2,NaN,NaN,180.1642,NaN,NaN,NaN,NaN
4,0,0,149,1697728200000,2023-10-19 15:10:00,EntryFilled,1180.1642,1120.9910,2947.671,59.174,...,3006.845,28703.5,NaN,0.105,3006.845,NaN,0.98,28420.8,2.4,29406.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0,0,2715,1698498000000,2023-10-28 13:00:00,EntryFilled,799.7465,597.0470,16348.384,202.699,...,5131.242,34035.7,NaN,0.493,16765.189,NaN,0.24,33930.5,1.0,34346.4
290,0,0,2992,1698581100000,2023-10-29 12:05:00,TakeProfitFilled,943.7933,943.7933,NaN,NaN,...,NaN,NaN,34346.4,NaN,NaN,144.0468,NaN,NaN,NaN,NaN
291,0,0,2994,1698581700000,2023-10-29 12:15:00,EntryFilled,943.7933,877.9650,4625.868,65.829,...,4691.697,34302.2,NaN,0.137,4691.697,NaN,0.42,34159.1,1.3,34752.9
292,0,0,2995,1698582000000,2023-10-29 12:20:00,EntryFilled,943.7933,820.9800,8467.281,122.814,...,3964.227,34340.0,NaN,0.252,8655.924,NaN,0.45,34159.1,1.4,34799.4
